In [ ]:
!pip install llama-index

In [ ]:
pip install openai

In [ ]:
!pip install neo4j


In [ ]:
import pandas as pd
import openai

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:

data = pd.read_csv("/content/gdrive/MyDrive/Q&A_model/dataframe_with_embeddings2.csv")

In [ ]:
len(data)

33

This code transforms our DataFrame into a list of Document objects, ready for indexing with llama_index. Each document contains the text passage, a unique id, and an extra field for the article title.

In [ ]:
from llama_index import Document


documents = []

for i, row in data.iterrows():
    documents.append(Document(
        text=row['text'],
        doc_id=row['file_chunks']

    ))
documents[0]

Document(id_='social.txt_1', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='b91b06bc97dbdd1471b42e3a928891613482bf1a542a2d98527503023a19e254', text='Page # 1\nMedia for Social Change: Producers, Viewers,\nContent\n\nTopic: Social Change | Words: 10292 Pages: 30 | Feb 12th, 2021\n\nIntroduction\n\nStatement of Research Issue\n\nThis research investigates media for social change may be used to create an environment where people can engage in\nthe public realm, mediate disagreements, and encourage civil debate. For an informative analysis, | have included my\npersonal view on media power and discussed communication as the main factor in creating hybrid public space, and\n\nencouraged the public to use media as an interactive tool in the field of social society.\n\nThe research proposes for reconsideration of media role in creating imaginative and narrative communication avenues\nin achieving a highly informed society. O

In [ ]:
len(docs)

NameError: ignored

Here, we're setting up the OpenAI API key and initializing a `SimpleNodeParser`. This parser processes our list of `Document` objects into 'nodes', which are the basic units that `llama_index` uses for indexing and querying. The first node is displayed below.

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = ""  # platform.openai.com

In [ ]:
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output

from llama_index import (
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext,
    SimpleDirectoryReader,
)
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import Neo4jGraphStore
from llama_index.llms import OpenAI

from IPython.display import Markdown, display


# define LLM
# NOTE: at the time of demo, text-davinci-002 did not have rate-limit errors
llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:


username = "neo4j"
password = ""
url = ""
database = "neo4j"

In [ ]:
graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
    service_context=service_context,
)

(Media, plays role in, creating)
(Media, influencing, opinion)
(Media, linking, ideas)
(producers, pass down, themes)
(producers, audience, perceive)
(ideas, audience, perceive)
(Alia, states, there exists a direct relationship between the change in the society and what the media producers are giving the society)
(media producers, generate, information from a chain of relations and influences)

(programs, assists, target audience)
(TV news programs, reflect, news' beliefs)
(producer, pass, ideas)
(producer, aim at, conceiving)
(Alia, observes, media functions as system of transmitting messages and representations)
(Ricoeur, sees, media as instruments for transmitting information, ideas, and opinions)
(media, has, ability)
(media, spreads, knowledge)
(media, has brought, social transformations)
(media, create awareness of, thorny issues)
(Ricoeur, According to, 1991)
(media, function as, instrument)
(media, formulate and evaluate, public opinions)
(media, link, world)
(media, link, thin

In [ ]:
query_engine = index.as_query_engine(include_text=False, response_mode="tree_summarize")

response = query_engine.query("Abusive supervision sleep patterns")

In [ ]:
print(response)

Abusive supervision can have a negative impact on sleep patterns.
